In [1]:
from dateutil.parser import parse
import email
import mailbox
import os
import re
import sys
from time import asctime

In [2]:
import pandas as pd

In [3]:
DIRNAME = os.path.dirname(os.getcwd())+os.sep
# Where to write the converted mbox
MBOX = DIRNAME+'enron'+os.sep+'enron_mail'+os.sep+'enron.mbox'

In [4]:
def maildir_to_mailbox(MAILDIR):
    with open(MBOX, 'w') as mbox:   
        for (root, dirs, file_names) in os.walk(MAILDIR):
            if root.split(os.sep)[-1].lower() != 'inbox':
                continue

            # Process each message in 'inbox'
            for file_name in file_names:
                file_path = os.path.join(root, file_name)
                message_text = open(file_path).read()

                # Compute fields for the From_ line in a traditional mbox message

                #_from = re.search(r"From: ([^\r]+)", message_text).groups()[0]
                #_date = re.search(r"Date: ([^\r]+)", message_text).groups()[0]

                # Convert _date to the asctime representation for the From_ line

                #_date = asctime(parse(_date).timetuple())

                msg = email.message_from_string(message_text)
                #msg.set_unixfrom('From %s %s' % (_from, _date))

                mbox.write(msg.as_string(unixfrom=True) + "\n\n")

    return mbox


In [19]:
maildir_to_mailbox(DIRNAME+'enron'+os.sep+'enron_mail'+os.sep+'maildir'+os.sep)

In [13]:
def mbox_to_df():
    """Convert the mailbox to a dataframe.

    Returns:
        dict: Enron corpus dataframe

    """
    mbox = mailbox.mbox(MBOX)

    mbox_dict = {}
    for i, msg in enumerate(mbox):
        mbox_dict[i] = {}
        for header in msg.keys():
            mbox_dict[i][header] = msg[header]
        mbox_dict[i]['Body'] = msg.get_payload().replace('\n', ' ').replace('\t', ' ').strip()

    df = pd.DataFrame.from_dict(mbox_dict, orient='index')
    df.index = df['Date'].apply(pd.to_datetime)

    return df

In [18]:
mbox_to_df()

KeyError: 'Date'